# Data Understanding

In [4]:
import pandas as pd

df = pd.read_csv("data/smoker_train.csv")

## Merkmale

### Leberwerte

In [5]:
df_urine = pd.read_csv("data/smoker_train.csv", usecols=["Urine protein"])
urine_counts = df_urine["Urine protein"].value_counts().sort_index()
print(urine_counts)

Urine protein
1    36836
2     1236
3      667
4      182
5       58
6        5
Name: count, dtype: int64
